In [1]:
# Install necessary libraries
!pip install diffusers accelerate transformers scipy ftfy bitsandbytes
!pip install datasets huggingface-hub
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118  # Ensure CUDA is enabled

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
from google.colab import drive
import os
import zipfile

# Mount Google Drive
drive.mount('/content/drive')

# Define paths
zip_path = '/content/drive/MyDrive/Stable_diffusion/dataset.zip'  # Adjust the path to your dataset.zip file
extract_to = '/content/drive/MyDrive/dataset'   # Define where to extract

# Ensure the destination folder exists
os.makedirs(extract_to, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Dataset extracted to {extract_to}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset extracted to /content/drive/MyDrive/dataset


In [10]:
# # Import necessary libraries
# from diffusers import StableDiffusionPipeline, UNet2DConditionModel, AutoencoderKL
# from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms
# from PIL import Image
# import torch
# import os
# import torch.nn.functional as F

# # Use the components directly from the Stable Diffusion pipeline
# pipe = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1-base", torch_dtype=torch.float16)

# # Extract components from the pipeline
# vae = pipe.vae
# unet = pipe.unet
# text_encoder = pipe.text_encoder
# tokenizer = pipe.tokenizer
# scheduler = pipe.scheduler

# # Rest of your dataset and training code remains the same
# # ... (keep the ImageCaptionDataset class as it is)

# # Ensure the model is moved to GPU if available
# device = "cuda" if torch.cuda.is_available() else "cpu"
# unet = unet.to(device)
# vae = vae.to(device)
# text_encoder = text_encoder.to(device)

# # Prepare dataset and dataloader
# dataset = ImageCaptionDataset(image_dir, caption_dir, tokenizer)
# dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# # Define optimizer and parameters
# optimizer = torch.optim.AdamW(unet.parameters(), lr=5e-6)

# # Training loop
# num_epochs = 10
# for epoch in range(num_epochs):
#     unet.train()
#     for batch in dataloader:
#         # Move data to device
#         pixel_values = batch["pixel_values"].to(device).half()
#         input_ids = batch["input_ids"].to(device)
#         attention_mask = batch["attention_mask"].to(device)

#         # Encode text and images
#         latents = vae.encode(pixel_values).latent_dist.sample() * 0.18215
#         encoder_hidden_states = text_encoder(input_ids, attention_mask=attention_mask).last_hidden_state

#         # Predict noise residual and compute loss
#         noise = torch.randn_like(latents).to(device)
#         timesteps = torch.randint(0, scheduler.config.num_train_timesteps, (latents.shape[0],), device=device).long()
#         noisy_latents = scheduler.add_noise(latents, noise, timesteps)

#         # Ensure hidden states are on the correct device and have the right dtype
#         encoder_hidden_states = encoder_hidden_states.to(device).half()

#         noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample
#         loss = F.mse_loss(noise_pred, noise)

#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1/10, Loss: nan
Epoch 2/10, Loss: nan


KeyboardInterrupt: 

In [11]:
# Save the fine-tuned model
# output_dir = "drive/MyDrive/stable_diffusion_finetuned"
# unet.save_pretrained(output_dir)
# vae.save_pretrained(output_dir)
# text_encoder.save_pretrained(output_dir)
# tokenizer.save_pretrained(output_dir)

('drive/MyDrive/stable_diffusion_finetuned/tokenizer_config.json',
 'drive/MyDrive/stable_diffusion_finetuned/special_tokens_map.json',
 'drive/MyDrive/stable_diffusion_finetuned/vocab.json',
 'drive/MyDrive/stable_diffusion_finetuned/merges.txt',
 'drive/MyDrive/stable_diffusion_finetuned/added_tokens.json')

In [ ]:
# Install required dependencies
!pip install -q torch transformers diffusers accelerate wandb bitsandbytes datasets peft

In [7]:
import numpy as np
import os
import torch.nn.functional as F
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from diffusers import StableDiffusionPipeline, UNet2DConditionModel
from peft import LoraConfig, get_peft_model
from PIL import Image

# Configuration
MODEL_NAME = "runwayml/stable-diffusion-v1-5"
DATASET_PATH = "/content/drive/MyDrive/dataset"
OUTPUT_DIR = "/content/drive/MyDrive/sd-lora-finetuned"
RESOLUTION = 512
TRAIN_BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = 4
LEARNING_RATE = 1e-4
MAX_TRAIN_STEPS = 1000

class CustomImageCaptionDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, caption_dir, tokenizer, size=512):
        self.image_dir = image_dir
        self.caption_dir = caption_dir
        self.tokenizer = tokenizer
        self.size = size

        # Collect image-caption pairs
        self.image_paths = []
        self.captions = []

        for img_file in os.listdir(image_dir):
            if img_file.endswith(('.png', '.jpg', '.jpeg')):
                caption_file = os.path.splitext(img_file)[0] + '.txt'
                caption_path = os.path.join(caption_dir, caption_file)

                if os.path.exists(caption_path):
                    with open(caption_path, 'r') as f:
                        caption = f.read().strip()

                    self.image_paths.append(os.path.join(image_dir, img_file))
                    self.captions.append(caption)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        image = image.resize((self.size, self.size))

        # Convert the PIL image to a NumPy array and then to a tensor
        image = torch.tensor(np.array(image)).permute(2, 0, 1).float() / 255.0

        # Tokenize caption
        text_inputs = self.tokenizer(
            self.captions[idx],
            max_length=self.tokenizer.model_max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            "pixel_values": image,
            "input_ids": text_inputs.input_ids.squeeze(),
            "attention_mask": text_inputs.attention_mask.squeeze()
        }

In [8]:
# Load base model and tokenizer
pipe = StableDiffusionPipeline.from_pretrained(MODEL_NAME)
tokenizer = pipe.tokenizer

# Prepare LoRA configuration
lora_config = LoraConfig(
    r=16,  # Rank
    lora_alpha=32,
    target_modules=["to_q", "to_v"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["unet"]
)

# Prepare UNet with LoRA
unet = pipe.unet
unet = get_peft_model(unet, lora_config)



Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [9]:
# Prepare dataset
train_dataset = CustomImageCaptionDataset(
    image_dir=f"{DATASET_PATH}/images",
    caption_dir=f"{DATASET_PATH}/captions",
    tokenizer=tokenizer
)

# DataLoader
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True
)



In [ ]:
# Optimizer
optimizer = torch.optim.AdamW(unet.parameters(), lr=LEARNING_RATE)

# Training Loop
unet.train()
for step, batch in enumerate(train_dataloader):
    if step >= MAX_TRAIN_STEPS:
        break

    # Forward pass
    latents = pipe.vae.encode(batch["pixel_values"]).latent_dist.sample()
    latents = latents * 0.18215

    # Prepare text embeddings
    encoder_hidden_states = pipe.text_encoder(batch["input_ids"])[0]

    # Add noise
    noise = torch.randn_like(latents)
    timesteps = torch.randint(0, pipe.scheduler.config.num_train_timesteps, (latents.shape[0],), device=latents.device).long()
    noisy_latents = pipe.scheduler.add_noise(latents, noise, timesteps)
    # Model prediction
    noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample
    optimizer.zero_grad()

    loss = F.mse_loss(noise_pred, noise)
    loss.backward()  # Backward pass
    optimizer.step()  # Optimizer step

    print(f"Step {step}/{MAX_TRAIN_STEPS}")

# Save the fine-tuned model
os.makedirs(OUTPUT_DIR, exist_ok=True)
unet.save_pretrained(OUTPUT_DIR)
print(f"Model saved to {OUTPUT_DIR}")

# Inference example after training
prompt = "A beautiful landscape"
pipe.unet.load_adapter(OUTPUT_DIR)
image = pipe(prompt).images[0]

# Save generated image
image.save("/content/generated_image.png")

Step 0/1000
Step 1/1000
Step 2/1000
Step 3/1000
Step 4/1000
Step 5/1000
Step 6/1000
Step 7/1000
Step 8/1000
Step 9/1000
Step 10/1000
Step 11/1000
Step 12/1000
Step 13/1000
Step 14/1000
Step 15/1000
Step 16/1000
Step 17/1000
Step 18/1000
Step 19/1000
Step 20/1000
Step 21/1000
